In [1]:
%load_ext memory_profiler

## Run benchmark with

From repo root/base folder

```bash
mprof run python extract_point_from_raster_buffer.py -f srg-dev/test-data/pop_density/pop_density/*.tif -g srg-dev/test-data/1000_testing_points.rds
```

## Run all the cells below to record peak memory and time

In [2]:
import pandas as pd
# import geopandas as gpd
import rioxarray as riox
import dask.dataframe as dd

from pyproj import Transformer
from shapely.geometry import mapping, Point

start = pd.Timestamp('now')
buffer_value = 10_000

## Run the analysis in a scalable way

load the raster and fill NaNs with 0

In [3]:
myraster = (
    riox.open_rasterio('test-data/pop_density/pop_density/apg18e_1_0_0_20210512.tif')
    .sel(band=1)
)

In [4]:
myraster = myraster.where(myraster != myraster.rio.nodata, drop=True)

In [5]:
"{:,}".format(myraster.data.shape[0] * myraster.data.shape[1])

'15,138,272'

Load points for data extraction and create buffers

In [6]:
transformer = Transformer.from_crs("EPSG:3577", myraster.rio.crs, always_xy=True)

In [7]:
points = (
    pd.read_csv('test-data/1000_testing_points.csv')
    .rename(columns={'X': 'x', 'Y': 'y'})
    .assign(
        lat_lon_tuple = lambda columns: columns[['x', 'y']].apply(lambda row: transformer.transform(row['x'], row['y']), axis=1),
        lat = lambda columns: columns['lat_lon_tuple'].apply(lambda el: el[0]),
        lon = lambda columns: columns['lat_lon_tuple'].apply(lambda el: el[1]),
        points = lambda columns: columns['lat_lon_tuple'].apply(Point),
        points_buffer = lambda columns: columns['points'].apply(lambda x: x.buffer(buffer_value))
    )
)
points = dd.from_pandas(points, npartitions=24)# assign partitions equal to 2 x Nr logical cores in my machine

Memory of dataframe in MB

In [8]:
def extract_mean_from_buffer(raster, geom):
    data_points = pd.Series(geom.exterior.coords)
    values_from_raster = data_points.apply(lambda row: raster.sel(x=row[0], y=row[1], method="nearest").item()).values
    return values_from_raster.mean()

In [9]:
points['extracted_mean'] = points['points_buffer'].apply(
    lambda x: extract_mean_from_buffer(myraster, x),
    meta=float
) * myraster.attrs['scale_factor'] + myraster.attrs['add_offset']

In [10]:
points = points.compute(scheduler="processes")

In [11]:
print(f"running time: {pd.Timestamp('now') - start}")

running time: 0 days 00:00:18.472120


In [12]:
points.sample(20)

,x,y,lat_lon_tuple,lat,lon,points,points_buffer,extracted_mean
311,863424.057862,-3.184429e+06,"(863424.057862312, -3184428.5805817773)",863424.057862,-3.184429e+06,POINT (863424.057862312 -3184428.580581777),"POLYGON ((873424.057862312 -3184428.580581777,...",0.000000
83,837509.458401,-3.184429e+06,"(837509.4584007077, -3184428.5805817773)",837509.458401,-3.184429e+06,POINT (837509.4584007077 -3184428.580581777),POLYGON ((847509.4584007077 -3184428.580581777...,0.000000
553,890929.904659,-3.184429e+06,"(890929.9046592778, -3184428.5805817773)",890929.904659,-3.184429e+06,POINT (890929.9046592778 -3184428.580581777),POLYGON ((900929.9046592778 -3184428.580581777...,0.000000
27,831144.469059,-3.184429e+06,"(831144.469059261, -3184428.5805817773)",831144.469059,-3.184429e+06,POINT (831144.469059261 -3184428.580581777),"POLYGON ((841144.469059261 -3184428.580581777,...",0.000000
973,938667.324720,-3.184429e+06,"(938667.3247201276, -3184428.5805817773)",938667.324720,-3.184429e+06,POINT (938667.3247201276 -3184428.580581777),POLYGON ((948667.3247201276 -3184428.580581777...,0.000000
411,874790.110258,-3.184429e+06,"(874790.1102577524, -3184428.5805817773)",874790.110258,-3.184429e+06,POINT (874790.1102577524 -3184428.580581777),POLYGON ((884790.1102577524 -3184428.580581777...,0.000000
530,888315.712608,-3.184429e+06,"(888315.7126083265, -3184428.5805817773)",888315.712608,-3.184429e+06,POINT (888315.7126083265 -3184428.580581777),POLYGON ((898315.7126083265 -3184428.580581777...,0.017554
864,926278.327609,-3.184429e+06,"(926278.3276090976, -3184428.5805817773)",926278.327609,-3.184429e+06,POINT (926278.3276090976 -3184428.580581777),POLYGON ((936278.3276090976 -3184428.580581777...,0.000000
533,888656.694180,-3.184429e+06,"(888656.6941801896, -3184428.5805817773)",888656.694180,-3.184429e+06,POINT (888656.6941801896 -3184428.580581777),POLYGON ((898656.6941801896 -3184428.580581777...,0.017554
659,902977.920198,-3.184429e+06,"(902977.9201984446, -3184428.5805817773)",902977.920198,-3.184429e+06,POINT (902977.9201984446 -3184428.580581777),POLYGON ((912977.9201984446 -3184428.580581777...,0.000000
